<a href="https://colab.research.google.com/github/elliot-brooks/nlu-coursework/blob/main/src/AV_LSTM_TRAIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
!pip install -U transformers
!pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 72.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 7.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import DistilBertTokenizer, TFDistilBertModel
import nltk
import re

# Load training data

In [3]:
training_corpus = pd.read_csv("train.csv", encoding='utf-8')

# Pre-process training data

In [4]:
# Case fold to lower-case
def preprocess(string):
  output = str(string).lower()
  return output

# Prepare data for Distilled Bert by concatenating pairs with [SEP] token
def prepare_data(data) :
  data["text_1"] = data["text_1"].apply(lambda x: preprocess(x))
  data["text_2"] = data["text_2"].apply(lambda x: preprocess(x))
  concat_pairs = []
  for index, row in data.iterrows():
      concatenated_pair = row["text_1"] + " [SEP] " + row["text_2"]
      concat_pairs.append(concatenated_pair)
  return concat_pairs

concat_data = prepare_data(training_corpus)

Create BERT embeddings

In [5]:
tokeniser = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
bert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [6]:
SEQ_LENGTH = 256
BATCH_SIZE = 32
def create_bert_embeddings_batch(texts, tokeniser, model, batch_size, seq_length) :
  embeddings = []
  for i in range(0, len(texts), batch_size) :
    batch = texts[i:i + batch_size]
    inputs = tokeniser.batch_encode_plus(batch, padding='max_length', truncation=True, return_tensors='tf', max_length=seq_length, add_special_tokens=True)

    # Create embeddings
    outputs = model(inputs['input_ids'], attention_mask=inputs['attention_mask'])

    last_hidden_state_CLS = outputs.last_hidden_state[:, 0, :]

    embeddings.append(last_hidden_state_CLS)
  return embeddings

bert_embeddings = create_bert_embeddings_batch(concat_data, tokeniser, bert_model, BATCH_SIZE, SEQ_LENGTH)

# Define classification model

In [8]:
LSTM_UNITS = 128
DROPOUT_RATE = 0.2
LEARNING_RATE = 0.001
model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(768, 1)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=LSTM_UNITS, recurrent_activation='sigmoid', return_sequences=False)),
        tf.keras.layers.Dropout(DROPOUT_RATE),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(DROPOUT_RATE),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(DROPOUT_RATE),
        tf.keras.layers.Dense(1, activation='sigmoid')
])


# Summarise Model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_1 (Bat  (None, 768, 1)            4         
 chNormalization)                                                
                                                                 
 bidirectional (Bidirection  (None, 256)               133120    
 al)                                                             
                                                                 
 dropout_19 (Dropout)        (None, 256)               0         
                                                                 
 dense (Dense)               (None, 64)                16448     
                                                                 
 dropout_20 (Dropout)        (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                2

# Train Model

In [9]:
train_inputs = np.concatenate(bert_embeddings, axis=0)
train_labels = np.array(training_corpus['label'])

print(train_inputs.shape)
print(train_labels.shape)
model.fit(train_inputs, train_labels, epochs=50, batch_size=512)

(30000, 768)
(30000,)
Epoch 1/50
59/59 [==============================] - 30s 111ms/step - loss: 0.6935 - accuracy: 0.4966
Epoch 2/50
59/59 [==============================] - 6s 110ms/step - loss: 0.6932 - accuracy: 0.4974
Epoch 3/50
59/59 [==============================] - 6s 110ms/step - loss: 0.6931 - accuracy: 0.5042
Epoch 4/50
59/59 [==============================] - 6s 110ms/step - loss: 0.6932 - accuracy: 0.5044
Epoch 5/50
59/59 [==============================] - 6s 110ms/step - loss: 0.6932 - accuracy: 0.5045
Epoch 6/50
59/59 [==============================] - 6s 110ms/step - loss: 0.6931 - accuracy: 0.5017
Epoch 7/50
59/59 [==============================] - 6s 110ms/step - loss: 0.6932 - accuracy: 0.5012
Epoch 8/50
59/59 [==============================] - 6s 110ms/step - loss: 0.6930 - accuracy: 0.5053
Epoch 9/50
59/59 [==============================] - 6s 110ms/step - loss: 0.6929 - accuracy: 0.5115
Epoch 10/50
59/59 [==============================] - 6s 110ms/step - loss: 0.

# Save Model

In [10]:
model.save("AV_LSTM_MODEL")

In [11]:
# Zip model file
!zip -r /content/LSTM_MODEL.zip /content/AV_LSTM_MODEL

  adding: content/AV_LSTM_MODEL/ (stored 0%)
  adding: content/AV_LSTM_MODEL/variables/ (stored 0%)
  adding: content/AV_LSTM_MODEL/variables/variables.index (deflated 65%)
  adding: content/AV_LSTM_MODEL/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: content/AV_LSTM_MODEL/fingerprint.pb (stored 0%)
  adding: content/AV_LSTM_MODEL/saved_model.pb (deflated 90%)
  adding: content/AV_LSTM_MODEL/keras_metadata.pb (deflated 91%)
  adding: content/AV_LSTM_MODEL/assets/ (stored 0%)
